<a href="https://colab.research.google.com/github/kdeary3/rank_detector/blob/main/rank_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running a YOLOv4 Object Detector with Darknet


In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14997, done.
remote: Total 14997 (delta 0), reused 0 (delta 0), pack-reused 14997
Receiving objects: 100% (14997/14997), 13.38 MiB | 23.43 MiB/s, done.
Resolving deltas: 100% (10194/10194), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# Download pre-trained YOLOv4 weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# Helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# Stops 'Run all'
assert False

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd darknet

#Importing into Darknet

In [ ]:
# Google Drive access
!ls /mydrive/CS485_project/

In [ ]:
# Copy Datasets
!cp /mydrive/CS485_project/obj.zip ../
!cp /mydrive/CS485_project/test.zip ../

In [ ]:
# Unzip Datasets
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

In [ ]:
# Upload custom cfg file
!cp /mydrive/CS485_project/yolov4-obj.cfg ./cfg

In [ ]:
# Upload the obj.names and obj.data
!cp /mydrive/CS485_project/obj.names ./data
!cp /mydrive/CS485_project/obj.data  ./data

In [ ]:
# Upload the generate_train.py and generate_test.py script
!cp /mydrive/CS485_project/generate_train.py ./
!cp /mydrive/CS485_project/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# train.txt and test.txt seen in darknet folder
!ls data/

In [ ]:
# Download pre-trained weights for Convolutional Layers
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Training



In [ ]:
# Train Rank Detector from stratch

!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# Show how detector did
imShow('chart.png')

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg content/drive/MyDrive/CS485_project/backup/yolov4-obj_last.weights -dont_show

In [ ]:
# mAP of Model
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/CS485_project/backup/yolov4-obj_2000.weights

#Run Object Detector

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# run your custom detector 
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/CS485_project/backup/yolov4-obj_2000.weights /content/drive/MyDrive/CS485_project/data/CPT/cpt3.jpg -thresh 0.5
imShow('predictions.jpg')